# Fractal Art

In [141]:
import csv

def parse_rules(input_path):
    enhancement_rules = {}
    symbol_dict = {'.': '0', '#': '1', '/': ''}
    with open(input_path, 'rt') as f_input:
        csv_reader = csv.reader(f_input, delimiter=' ')
        for line in csv_reader:
            key = ''.join(list(map(lambda a: symbol_dict[a], list(line[0]))))
            value = ''.join(list(map(lambda a: symbol_dict[a], list(line[2]))))
            enhancement_rules[key] = value
    return enhancement_rules

## Part 1

In [150]:
import numpy as np
from itertools import product

def make_fractal(seq):
    n = int(np.sqrt(len(seq)))
    return np.array(list(seq), dtype=np.object).reshape((n, n))

def make_seq(fractal):
    return ''.join(list(fractal.reshape(fractal.shape[0] ** 2)))

def three_index_rotate(i, j):
    z = np.array([i - 1, j - 1])
    m = np.array([[0, 1], [-1, 0]])
    return tuple(m.dot(z))

def rotate(fractal):
    if fractal.shape[0] == 3:
        rot_fractal = np.empty((3, 3), dtype=np.object)
        for i, j in product([0, 1, 2], repeat=2):
            if i == j == 1:
                rot_fractal[i, j] = fractal[i, j]
            else:
                a, b = three_index_rotate(i, j)
                rot_fractal[a + 1, b + 1] = fractal[i, j]
        return rot_fractal
    elif fractal.shape[0] == 2:
        rot_fractal = np.empty((2, 2), dtype=np.object)
        rot_dict = {(0, 0): (0, 1), (0, 1): (1, 1), (1, 1): (1, 0), (1, 0): (0, 0)}
        for i, j in product([0, 1], repeat=2):
            rot_fractal[rot_dict[(i, j)]] = fractal[i, j]
        return rot_fractal

def v_flip(fractal):
    flipped = np.empty_like(fractal)
    if flipped.shape[0] == 3:
        flipped[1,:] = fractal[1,:]
    flipped[0,:]= fractal[-1,:]
    flipped[-1,:]= fractal[0,:]
    return flipped

def h_flip(fractal):
    flipped = np.empty_like(fractal)
    if flipped.shape[0] == 3:
        flipped[:,1] = fractal[:,1]
    flipped[:,0]= fractal[:,-1]
    flipped[:,-1]= fractal[:,0]
    return flipped
    
def generate_all_group(fractal):
    # TODO: create a generator that does
    # BFS by avoiding repeats
    queue = [fractal]
    visited = []
    while queue:
        f = queue.pop(0)
        f_hash = hash(f.tostring())
        if f_hash not in visited:
            visited.append(f_hash)
            queue.append(rotate(f))
            queue.append(v_flip(f))
            queue.append(h_flip(f))
            yield f
    
def enhance(fractal, er):
    seq = make_seq(fractal)
    for f in generate_all_group(fractal):
        seq = make_seq(f)
        if seq in er:
            return make_fractal(er[seq])
    else:
        print('did not match any enhancement rule')

def transform(fractal, er):
    n = fractal.shape[0]
    if n % 2 == 0:
        transformed = np.empty((3 * n // 2, 3 * n // 2), dtype=np.object)
        for i, j in product(range(n // 2), repeat=2):
            transformed[3 * i: 3 * (i + 1), 3 * j: 3 * (j + 1)] = enhance(fractal[2 * i: 2 * (i + 1), 2 * j: 2 * (j + 1)], er)
    elif n % 3 == 0:
        transformed = np.empty((4 * n // 3, 4 * n // 3), dtype=np.object)
        for i, j in product(range(n // 3), repeat=2):
            transformed[4 * i: 4 * (i + 1), 4 * j: 4 * (j + 1)] = enhance(fractal[3 * i: 3 * (i + 1), 3 * j: 3 * (j + 1)], er)
    return transformed

def main_iterate(initial_state, n, er):
    fractal = make_fractal(initial_state)
    for _ in range(n):
        fractal = transform(fractal, er)
    return fractal

def num_ones(fractal):
    return sum(fractal[i, j] == '1' for i, j in product(range(fractal.shape[0]), repeat=2))

### Test

In [151]:
er_test = {'0001': '110100000', '010001111': '1001000000001001'}
fractal = main_iterate('010001111', 2, er_test)
num_ones(fractal)

12

### Solution

In [152]:
er_sol = parse_rules('input.txt')
fractal = main_iterate('010001111', 5, er_sol)
num_ones(fractal)

152

## Part 2

In [153]:
er_sol = parse_rules('input.txt')
fractal = main_iterate('010001111', 18, er_sol)
num_ones(fractal)

1956174